# Customer Interest Clustering for Marketing

This is a tutorial on how to perform customer clustering based on the interests and purchases of customers. 

Marketing teams frequently are interested in this analysis.

We'll show how using graph analytics can be used to gain insights about the interests of customers by finding communities of customers who've bought similar products. 

We'll accomplish this by creating a bipartite graph of customers and products, using a graph projection to create a graph of customers linked to other customers who've bought the same product, and using Louvain community detection to find the communities.

We'll be using ecommerce transaction data from a U.K. retailer provided by the University of California, Irvine. The data can be found [here](https://www.kaggle.com/carrie1/ecommerce-data).

# Data Preprocessing

Let's first look at the data.

First, we'll need to import some libraries.

In [1]:
import metagraph as mg
import pandas as pd
import networkx as nx

Let's see what the data looks like.

In [2]:
RAW_DATA_CSV = './data.csv' # https://www.kaggle.com/carrie1/ecommerce-data
data_df = pd.read_csv(RAW_DATA_CSV, encoding="ISO-8859-1")
data_df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom

Let's clean the data to make sure there aren't any missing values. 

In [3]:
data_df.InvoiceDate = pd.to_datetime(data_df.InvoiceDate, format="%m/%d/%Y %H:%M")
data_df.drop(data_df.index[data_df.CustomerID != data_df.CustomerID], inplace=True)
assert len(data_df[data_df.isnull().any(axis=1)])==0, "Raw data contains NaN"
data_df = data_df.astype({'CustomerID': int}, copy=False)
data_df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55       17850  United Kingdom  
1 2010-12-01 08:26:00       3.39       17850  United Kingdom  
2 2010-12-01 08:26:00       2.75       17850  United Kingdom  
3 2010-12-01 08:26:00       3.39       17850  United Kingdom  
4 2010-12-01 08:26:00       3.39       17850  United Kingdom

Note that some of these transactions are for returns (denoted by negative quantity values).

In [4]:
data_df[data_df.Quantity < 1].head()

InvoiceNo StockCode                       Description  Quantity  \
141   C536379         D                          Discount        -1   
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   

            InvoiceDate  UnitPrice  CustomerID         Country  
141 2010-12-01 09:41:00      27.50       14527  United Kingdom  
154 2010-12-01 09:49:00       4.65       15311  United Kingdom  
235 2010-12-01 10:24:00       1.65       17548  United Kingdom  
236 2010-12-01 10:24:00       0.29       17548  United Kingdom  
237 2010-12-01 10:24:00       0.29       17548  United Kingdom

Though customers may have returned these products, they did initially purchase the products (which reflects an interest in the product), so we'll keep the initial purchases. However, we'll remove the return transactions (which will also remove any discount transactions as well). 

In [5]:
data_df.drop(data_df.index[data_df.Quantity <= 0], inplace=True)
data_df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55       17850  United Kingdom  
1 2010-12-01 08:26:00       3.39       17850  United Kingdom  
2 2010-12-01 08:26:00       2.75       17850  United Kingdom  
3 2010-12-01 08:26:00       3.39       17850  United Kingdom  
4 2010-12-01 08:26:00       3.39       17850  United Kingdom

# Community Detection

Let's now find the communities of customers with similar purchases / interests. 

First, we'll need to create a bipartite graph of customers and products. 

In [6]:
bipartite_graph = nx.from_pandas_edgelist(data_df, 'CustomerID', 'StockCode')

Next, we'll need to use a graph projection to create a graph of customers linked to other customers who've bought the same product.

In [7]:
customer_similarity_graph = nx.projected_graph(bipartite_graph, data_df.CustomerID.unique())

Now, we'll need to use Louvain community detection to find similar communities based on purchased products.

In [8]:
r = mg.resolver
customer_similarity_graph_wrapped = r.wrappers.EdgeMap.NetworkXEdgeMap(customer_similarity_graph)
labels, modularity_score = r.algos.clustering.louvain_community(customer_similarity_graph_wrapped)

Let's now merge the labels into our dataframe.

In [9]:
data_df['CustomerCommunityLabel'] = data_df.CustomerID.map(lambda customer_id: labels.value[customer_id])
data_df.sample(10)

InvoiceNo StockCode                     Description  Quantity  \
366736    568791     22179         SET 10 NIGHT OWL LIGHTS         2   
452094    575322     21889          WOODEN BOX OF DOMINOES        18   
99049     544692    84406B  CREAM CUPID HEARTS COAT HANGER         4   
485779    577669     22762   CUPBOARD 3 DRAWER MA CAMPAGNE         1   
315010    564715     22030          SWALLOWS GREETING CARD        12   
230147    557123     22551        PLASTERS IN TIN SPACEBOY         4   
237143    557802     21677                HEARTS  STICKERS        12   
308387    563952    82494L     WOODEN FRAME ANTIQUE WHITE          8   
285501    561910     21843        RED RETROSPOT CAKE STAND         6   
294263    562708     21621      VINTAGE UNION JACK BUNTING         3   

               InvoiceDate  UnitPrice  CustomerID         Country  \
366736 2011-09-29 09:51:00       6.75       16650  United Kingdom   
452094 2011-11-09 13:36:00       1.25       14543  United Kingdom   
99049  2011-02-23 09:14:00       4.15       17499  United Kingdom   
485779 2011-11-21 10:48:00      14.95       15567  United Kingdom   
315010 2011-08-28 11:01:00       0.42       14472  United Kingdom   
230147 2011-06-16 17:56:00       1.65       15555  United Kingdom   
237143 2011-06-23 10:15:00       0.85       15130  United Kingdom   
308387 2011-08-22 10:44:00       2.95       15572  United Kingdom   
285501 2011-08-01 10:23:00       9.95       17053  United Kingdom   
294263 2011-08-08 14:15:00       8.50       17085  United Kingdom   

        CustomerCommunityLabel  
366736                       3  
452094                       2  
99049                        3  
485779                       0  
315010                       0  
230147                       3  
237143                       0  
308387                       0  
285501                       0  
294263                       0

We now have clusters of customers who've bought similar products and can market to these interests. 